<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_nlp_projects/blob/main/News_group/New_group_classification_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/Data/news_group"

/content/drive/MyDrive/Data/news_group


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

## Clean train and test data

use the cleaned train and test data

In [4]:
!pip install transformers[torch] datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.6 MB/s eta 0:00:00


In [5]:
from sklearn.model_selection import train_test_split

from transformers import (AutoTokenizer,
                          AdamW,
                          AutoModelForSequenceClassification)

from datasets import load_dataset

import torch, gc
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn

import evaluate

from tqdm.notebook import tqdm

In [6]:
def seed_everything(seed: int = 42):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed= 42
seed_everything(seed)

In [15]:
test = pd.read_csv('./test.csv')
test.head()

,id,text
0,0,\nThe VL-IDE Adapter can be much faster then t...
1,1,\n\nYeah. In a fire that reportedly burned ho...
2,2,":Judge: ""I grant you immunity from whatever ma..."
3,3,"I, too, put a corbin seat on my Hawk. I got t..."
4,4,\n\nDo I ever!!!!!! After 2 years of having h...


In [16]:
import re
# 숫자와 영문 빼고 모두 제거
def clean_text(texts):
  corpus = []
  for i in range(0, len(texts)):

    review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\]\[\>]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거
    review = re.sub(r'\s+', ' ', review) #extra space 제거
    review = re.sub(r'<[^>]+>','',review) #Html tags 제거
    review = re.sub(r'\s+', ' ', review) #spaces 제거
    review = re.sub(r"^\s+", '', review) #space from start 제거
    review = re.sub(r'\s+$', '', review) #space from the end 제거
    review = re.sub(r'_', ' ', review) #space from the end 제거
    corpus.append(review)

  return corpus

In [17]:
temp =clean_text(test['text'])
test['text'] = temp
test.head()

,id,text
0,0,The VLIDE Adapter can be much faster then the ...
1,1,Yeah In a fire that reportedly burned hotter t...
2,2,Judge I grant you immunity from whatever may b...
3,3,I too put a corbin seat on my Hawk I got the s...
4,4,Do I ever After 2 years of having health probl...


In [7]:
bert_name = "bert-base-uncased"
electra_name = 'google/electra-base-discriminator'
roberta_name = 'xlm-roberta-base'

bert_tokenizer = AutoTokenizer.from_pretrained(bert_name)
electra_tokenizer = AutoTokenizer.from_pretrained(electra_name)
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_name)

In [8]:
max_len = 512

class CustomDataset(Dataset):

  def __init__(self, dataset, tokenizer, train_mode = True):
    self.dataset = dataset
    self.train_mode = train_mode
    self.tokenizer= tokenizer

  def __getitem__(self, index):
    text = self.dataset.loc[index, 'text']

    encoded_data = self.tokenizer.encode_plus(
        text,
        add_special_tokens = True,
        max_length = max_len,
        padding = 'max_length',
        truncation = True,
        return_attention_mask = True,
        return_tensors ='pt'
        )
    input_ids = encoded_data['input_ids'][0]
    attention_masks = encoded_data['attention_mask'][0]

    if self.train_mode:
      labels = torch.tensor(self.dataset.loc[index, 'target'])
      return input_ids, attention_masks, labels
    return input_ids, attention_masks

  def __len__(self):
    return len(self.dataset)

In [9]:
BATCH_SIZE = 16

In [10]:
def compute_metrics(pred, label):
  return sum(pred==label)/len(label)

In [11]:
id2label = {0 : 'alt.atheism', 1: 'comp.graphics',2 : 'comp.os.ms-windows.misc',
           3 : 'comp.sys.ibm.pc.hardware', 4 : 'comp.sys.mac.hardware',
           5 : 'comp.windows.x', 6 : 'misc.forsale', 7: 'rec.autos',
           8 : 'rec.motorcycles', 9 : 'rec.sport.baseball', 10 : 'rec.sport.hockey',
           11 : 'sci.crypt', 12 : 'sci.electronics', 13 : 'sci.med',
           14 : 'sci.space', 15 : 'soc.religion.christian', 16 : 'talk.politics.guns',
           17 : 'talk.politics.mideast', 18 : 'talk.politics.misc', 19 : 'talk.religion.misc'}


label2id = {'alt.atheism': 0, 'comp.graphics': 1, 'comp.os.ms-windows.misc': 2,
            'comp.sys.ibm.pc.hardware': 3, 'comp.sys.mac.hardware': 4, 'comp.windows.x': 5,
            'misc.forsale': 6, 'rec.autos': 7, 'rec.motorcycles': 8, 'rec.sport.baseball': 9,
            'rec.sport.hockey': 10, 'sci.crypt': 11, 'sci.electronics': 12, 'sci.med': 13,
            'sci.space': 14, 'soc.religion.christian': 15,'talk.politics.guns': 16,
            'talk.politics.mideast': 17, 'talk.politics.misc': 18, 'talk.religion.misc': 19}


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Bert** "bert-base-uncased"
- Learning_rate =  2e-5
- Epoch:3, Train Loss: 0.451, Validation Loss: 0.930, Accuracy: 0.733

**Electra** 'google/electra-base-discriminator'
- Learning_rate =  2e-5
- Epoch:4, Train Loss: 0.517, Validation Loss: 0.988, Accuracy: 0.720


**Roberta** 'xlm-roberta-base'
- Learning_rate =  2e-5
- Epoch:3, Train Loss: 0.653, Validation Loss: 0.961, Accuracy: 0.718


In [13]:
def predict(model_name, model_load, testloader):
  model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=20,
                                                           id2label=id2label,
                                                           label2id=label2id).to(device)

  model.load_state_dict(torch.load(model_load, map_location=device))
  model.eval()
  probs = None
  with torch.no_grad():
    for input_ids, attention_mask in tqdm(testloader):
      input_ids = input_ids.to(device)
      attention_mask = attention_mask.to(device)

      outputs = model(input_ids, attention_mask)[0]
      outputs = outputs.cpu().numpy()
      if probs is None:
        probs = outputs
      else:
        probs = np.concatenate([probs, outputs])
  return probs

In [18]:
test_set = test.reset_index(drop=True)
test_set.head()

,id,text
0,0,The VLIDE Adapter can be much faster then the ...
1,1,Yeah In a fire that reportedly burned hotter t...
2,2,Judge I grant you immunity from whatever may b...
3,3,I too put a corbin seat on my Hawk I got the s...
4,4,Do I ever After 2 years of having health probl...


In [19]:
bert_dataset = CustomDataset(test_set, bert_tokenizer, train_mode=False)
electra_dataset = CustomDataset(test_set, electra_tokenizer, train_mode=False)
roberta_dataset = CustomDataset(test_set, roberta_tokenizer, train_mode=False)



bert_testloader = DataLoader(bert_dataset, batch_size=BATCH_SIZE, shuffle=False)
electra_testloader = DataLoader(electra_dataset, batch_size=BATCH_SIZE, shuffle=False)
roberta_testloader = DataLoader(roberta_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [20]:
bert_model_load = './Bert.model'
electra_model_load = './Electra.model'
roberta_model_load = './Roberta.model'

prod1 = predict(bert_name, bert_model_load, bert_testloader)
prod2 = predict(electra_name, electra_model_load, electra_testloader)
prod3 = predict(roberta_name, roberta_model_load, roberta_testloader)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/578 [00:00<?, ?it/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/578 [00:00<?, ?it/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/578 [00:00<?, ?it/s]

In [21]:
submission = pd.read_csv('./sample_submission.csv')
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [23]:
probs = prod1 + prod2 + prod3
_, preds = torch.max(torch.tensor(probs), dim=1)

In [24]:
submission['target'] = preds
submission.head()

,id,target
0,0,3
1,1,16
2,2,11
3,3,8
4,4,12


In [25]:
submission.to_csv('./submission_ensemble.csv', index=False)